In [29]:
from matplotlib import pyplot as plt

import numpy as np

#연소실에서의 "대류" 열전달 계산

def delta_I(I0, I_wall, x):
    return I0(x) - I_wall(x)

def D(D_wall, D_t):
    return D_wall / D_t

def psi(Mach_gas, r, Gamma, T_wall, T_0):
    psi = (2 * np.arctan(Mach_gas * np.sqrt(r * (Gamma - 1) / 2)) / ((np.sqrt(T_wall / T_0) + 1) * Mach_gas * np.sqrt(r * (Gamma - 1) / 2))) ** 2
    return psi

def Re_T00(x1, x2, m, B, Re_0, Pr_gas, n, Gamma,
           mu_wall, Mu, D, delta_I, psi, Re_T0, num_points=1000):

    x_vals = np.linspace(x1, x2, num_points)
    dx = (x2 - x1) / (num_points - 1)
    integral_sum = 0

    for x in x_vals:
        mu_ratio = mu_wall(x) / Mu
        D_val = D(x)
        dI = delta_I(x)
        PSI = psi(x)

        integrand = (
            PSI *
            mu_ratio ** m *
            np.sqrt((Gamma - 1) / (Gamma + 1)) *
            (2 / (Gamma + 1)) ** (1 / (Gamma - 1)) *
            D_val ** (m + 1) *
            dI
        )
        integral_sum += integrand * dx

    prefactor = (1 + m) * B * Re_0 / (2 * Pr_gas ** n)
    bracket_term = prefactor * integral_sum + (Re_T0 * delta_I(x1) * D(x1)) ** (1 + m)

    final_result = (1 / (delta_I(x2) * D(x2))) * (bracket_term) ** (1 / (1 + m))
    return final_result

def St(psi, B, Re_T0, Pr_gas, n, mu_wall, Mu, m):
    return psi * (B / (2 * Re_T0 * Pr_gas ** n)) * (mu_wall / Mu) ** m

def q_conv(St , density_0 , omega_0 , delta_I):
      q_conv = St*density_0*omega_0*delta_I
      return q_conv


#냉각수의 온도 계산
def Q_conv(m_dot_w, Cp, T_wi, T_wi_1):
    return m_dot_w * Cp * (T_wi - T_wi_1)


#냉각채널 내부 연소실 쪽 벽면 온도 계산
def Z(lambda_w, Cp_c, mu_c):
    return (lambda_w ** 0.57) * (Cp_c ** 0.43) / (mu_c ** 0.37)

def psi_coolant(mu_c, Cp_c, lambda_w, mu_wall, Cp_wall, lambda_wall):
    Pr = mu_c * Cp_c / lambda_w
    Pr_wall = mu_wall * Cp_wall / lambda_wall
    return (Pr / Pr_wall) ** 0.25

def alpha(m_dot_w, A_w, D_c, Z, psi_cool):
    return (0.021 / D_c ** 0.2) * (m_dot_w / A_w) ** 0.8 * Z * psi_cool #psi_coolant라고쓰면 안된다는데 나중에 검증할예정

def q_conv_wall(alpha, T_wall_wall, T_coolant):
    return alpha * (T_wall_wall - T_coolant)



Pr_gas = 0.8      #가스의 Pr계수
Mach_gas = 1      #가스의 마하수
Cp_gas = 2289.3   #가스의 정압비열
T_0 = 2981.22     #가스의 전온도 [K]
P_0 = 20e5        #가스의 전압력 [pa]
Gamma = 1.2107    #가스의 비열비
M_dot = 0.45      #가스의 질유량
Mu = 7.78e-05     #가스의 점성계수
w = 0.6           #물성지수성지수
density_0 = 1.68  # 연소가스의 밀도
omega_0 = 1199.2  #마하1로 가정하였을때의 가스 속도

#구조 성분

Wall_Thick = 2*1e-3 #[m]
k_w = 120             #벽 열전도율

#area = 1.398e-3     #[m^2]
D_t = 19.8e-3 # 목 직경
A_t = (np.pi * D_t**2)/4    # 목 면적
R = 0             #곡률반지름



#냉각체널 면적(case 1)
A_w = 4e-6


#냉각제 성분

T_c = 291.7           # 냉각제 입구 온도 (초기)
k_c = 0.6            # 냉각제 열전도율
m_dot_w = 1
Cp_c = 4182         # 냉각제 정압비열  [J/kg·K]
Mu_c = 9.57e-4          # 냉각제 점성계수
Rho_c = 996.9         # 냉각제 밀도
U_c = 31.93            # 냉각제 속도
